In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/19 10:05
# @Author  : Wang Yujia
# @File    : data_extract_for_targets.ipynb
# @Description : 为了target data提取信息，而且提取data_key. 然后重新设置一下粒度。

# 1. Preparations
## 1.1 全局设置
1. input的data包括那些duration过长的结果
2. output一共`data_key.shape[0]`+1个文件

In [1]:
data_path = "../data/datawithnp_2_selected.csv"
prod_embedding_output_path = "../data/prod_embedding.csv"

# output path
# target_output_head= "../data/targets_5/target_data_NP_"
target_output_head= "../data/targets_5_DA/target_data_NP_"
target_output_tail= ".csv"
target_data_key_path = "../data/target_datakey.csv"

features = ['id', 'bidincrement', 'bidfee','retail']
import pandas as pd
import numpy as np
from visdom import Visdom
import csv
import os
from tqdm import tqdm
from IPython.display import clear_output

## 1.2 data读取与保存data_key

In [3]:
data = pd.read_csv(data_path, encoding="utf-8")
data.head(),data.shape

(   auction_id  product_id                                      item  \
 0       86827    10009602  sony-ericsson-s500i-unlocked-mysterious-   
 1       87964    10009881            psp-slim-lite-sony-piano-black   
 2       87965    10009881            psp-slim-lite-sony-piano-black   
 3       88638    10006115  sony-ericsson-s500i-unlocked-mysterious-   
 4       88639    10006115  sony-ericsson-s500i-unlocked-mysterious-   
 
                                             desc  retail  price  finalprice  \
 0  Sony Ericsson S500i Unlocked Mysterious Green  499.99  13.35       13.35   
 1               PSP Slim & Lite Sony Piano Black  169.99  74.70       74.70   
 2               PSP Slim & Lite Sony Piano Black  169.99  83.10       83.10   
 3  Sony Ericsson S500i Unlocked Mysterious Green  499.99  19.65       19.65   
 4  Sony Ericsson S500i Unlocked Mysterious Green  499.99  47.10       47.10   
 
    bidincrement  bidfee        winner  ...           endtime_str  \
 0          0.1

In [4]:
data_key = data[features].copy()
data_key.drop_duplicates(inplace=True,ignore_index=True)
# make sure int
data_key['id'] = data_key['id'].astype(int)
data_key.to_csv(target_data_key_path,header=True, encoding="utf-8",index=False)
print(f"target data中，一共包括 *{data_key.shape[0]}* 个data keys")
data_key.head(),data_key.shape

target data中，一共包括 *1196* 个data keys


(   id  bidincrement  bidfee  retail
 0   0          0.15    0.75  499.99
 1   1          0.15    0.75  169.99
 2   2          0.15    0.75  299.99
 3   3          0.15    0.75   89.99
 4   5          0.15    0.75   59.99,
 (1196, 4))

## 1.3 functions about 'key'

In [5]:
# get key from i in 'data_key'
def get_key_from_index(i,flag="NotStr"):
    if(flag == "str"):
        key_i = list(data_key.iloc[i,:])
        key_i_str = (str(int(key_i[0])),str(key_i[1]),str(key_i[2]),str(key_i[3]))   # 'product_id'是int类型
        return key_i_str
    else:
        key_i = data_key.iloc[i,:]
        return key_i

def select_data_fromkey(key_i_str):
    """
        根据key_i_str从data中提取target data

    """
    return data[(data['id'] == key_i_str[0]) & (data['bidincrement'] == key_i_str[1]) & (data['bidfee'] == key_i_str[2]) & (data['retail'] == key_i_str[3])].copy()

# 2. 整理与输出

1. 整理成一个[key]对应一个“N” vector的格式
    - 最后的data中不包括"cnt_n_2"这一列，
2. 注意一定要按照`data_key`逐行保存csv，这样index可以对应


In [17]:
plot_flag = 0
if(plot_flag):
    env_str = 'N_targets'
    viz = Visdom(env=env_str)

# col_names = ['id','bidincrement','bidfee','retail','N','P','cnt_n_2']
# Col we want to save in target file
col_names_3 = ['N','cnt_n_2','P']
col_names_2 = ['N', 'P']
col_names_4 = ['N','cnt','P']

data_key.shape[0]

## 2.1 整理粒度
1. 添加了“整理粒度”的代码，注意必须有'cnt'这个属性才可以进行粒度的整理

In [7]:
# 复制training data文件夹下的第i个data到新路径
def save_i_trainfile(i):
    train_path = r"../data/train"
    train_path_2 = r"../data/train_2"

    train_all_path = os.listdir(train_path)

    train_path_i_path = os.path.join(train_path,train_all_path[i])
    train_path_i_path_2 = os.path.join(train_path_2,train_all_path[i])

    train_df = pd.read_csv(train_path_i_path,encoding="utf-8")
    train_df.to_csv(train_path_i_path_2,header=True,index=False,encoding="utf-8")

- 粒度为2：对于`data_lenth >= 60`的target data进行改动

In [44]:
# 粒度为2
# data_key.shape[0]
for i in tqdm(range(0,data_key.shape[0])):
# for i in tqdm(range(0,10)):

    # Get i_th data_key
    key_i = get_key_from_index(i)
    # Get data according to the data_key
    data_i = select_data_fromkey(key_i)
    # Sort data according to 'N'
    data_i.reset_index(drop=True,inplace=True)
    data_i.sort_values(by='N',ignore_index=True,inplace=True)
    # Only save col we need
    data_i_df = data_i[col_names_3]

    # 现在数据是无重复版本的，用'cnt_n_2'记录了这个record重复出现的次数
    # New added: Rearrange the data according to their length
    data_lenth = data_i_df.shape[0]         # Data length
    dele_idx = []                           # Idx to be deleted
    if(data_lenth >= 0):
        j = 0
        while (j < data_lenth):
            # 如果N值是奇数
            if(data_i_df.iloc[j,0]%2 != 0):
                # i不是data最后一个（奇数）值并且i+1的N值紧挨着i
                if( (j < data_lenth-1) and (data_i_df.iloc[j,0] == (data_i_df.iloc[j+1,0]-1))):
                    # prob值和cnt值进行叠加
                    data_i_df.iloc[j,1] += data_i_df.iloc[j+1,1]
                    data_i_df.iloc[j,2] += data_i_df.iloc[j+1,2]
                    # 删除[i+1]
                    dele_idx.append(j+1)
                    j += 2
                # i是data最后一个（奇数）或者[i+1]的值没什么关系
                else:
                    j += 1
            # 如果N值是偶数并且不存在[i-1]和[i]存在N值相邻的关系
            else:
                # 直接把N值改为奇数
                data_i_df.iloc[j,0] -= 1
                j += 1

    ######### 如果只想保存改动的data，把下面都拿到上面的if里面来
    # Save i_th training data file
    # save_i_trainfile(i)
    # Dele
    data_i_df = data_i_df.drop(dele_idx,axis = 0).copy()

    # Reconstruct and repeat data according to 'cnt_n_2'
    data_i_df = data_i_df.loc[data_i_df.index.repeat(data_i_df['cnt_n_2'])]
    data_i_df = data_i_df[col_names_2]

    # Get the output path
    target_output_path = target_output_head+str(i).zfill(4)+target_output_tail
    data_i_df.to_csv(target_output_path,header=True,index=False,encoding="utf-8")

# clear_output()
print("Done")

100%|██████████| 1196/1196 [00:10<00:00, 110.95it/s]

Done


- 粒度为4:
- N值只会有3种情况：恰好是4倍且（一定）位于开头；不是4倍且被并入某一个开头；不是4倍且未被并入任何一个已存在的开头（此时需要create一个开头）

In [10]:
# 粒度为4
for i in tqdm(range(0,data_key.shape[0])):
# for i in tqdm(range(0,10)):

    # Get i_th data_key
    key_i = get_key_from_index(i)
    # Get data according to the data_key
    data_i = select_data_fromkey(key_i)
    # Sort data according to 'N'
    data_i.reset_index(drop=True,inplace=True)
    data_i.sort_values(by='N',ignore_index=True,inplace=True)
    # Only save col we need
    data_i_df = data_i[col_names_3]

    # 现在数据是无重复版本的，用'cnt_n_2'记录了这个record重复出现的次数
    # New added: Rearrange the data according to their length
    data_lenth = data_i_df.shape[0]         # Data length
    dele_idx = []                           # Idx to be deleted

    if(data_lenth >= 0):
        j = 0
        while (j < data_lenth):
            # N值-1恰好是4倍
            if((data_i_df.iloc[j,0]-1)%4 == 0):
                k = j+1    # 用k记录位置
                # k不是data最后一个值 and [k]在+4的范围内
                while((k < data_lenth) and (data_i_df.iloc[j,0]+4 > data_i_df.iloc[k,0])):
                    # 更新[j]的prob值和cnt值：叠加
                    data_i_df.iloc[j,1] += data_i_df.iloc[k,1]
                    data_i_df.iloc[j,2] += data_i_df.iloc[k,2]
                    # 删除[k]
                    dele_idx.append(k)
                    k += 1
                # j 从 k（其实是k+1）的位置继续
                j = k
            # N值不是4倍且未被并入任何一个已存在的开头
            else:
                # 原地修改（减小）成一个新的区间的开头,
                while((data_i_df.iloc[j,0]-1) %4 != 0):
                    data_i_df.iloc[j,0] -= 1
                # 注意不需要j+1，下一次循环从当前开始，check后面的n需不需要并进来
                # j += 1

    ######### 如果只想保存改动的data，把下面都拿到上面的if里面来
    # Save i_th training data file
    # save_i_trainfile(i)
    # Dele
    data_i_df = data_i_df.drop(dele_idx,axis = 0).copy()

    # Reconstruct and repeat data according to 'cnt_n_2'
    data_i_df = data_i_df.loc[data_i_df.index.repeat(data_i_df['cnt_n_2'])]
    data_i_df = data_i_df[col_names_2]

    # Get the output path
    target_output_path = target_output_head+str(i).zfill(4)+target_output_tail
    data_i_df.to_csv(target_output_path,header=True,index=False,encoding="utf-8")

# clear_output()
print("Done")

100%|██████████| 1196/1196 [00:15<00:00, 76.76it/s] 

Done


### 2.1.2 粒度=5
- 方法一：仿照4直接改

In [12]:
# 粒度为5
for i in tqdm(range(0,data_key.shape[0])):
# for i in tqdm(range(0,10)):

    # Get i_th data_key
    key_i = get_key_from_index(i)
    # Get data according to the data_key
    data_i = select_data_fromkey(key_i)

    # Sort data according to 'N'
    data_i.reset_index(drop=True,inplace=True)
    data_i.sort_values(by='N',ignore_index=True,inplace=True)
    # Only save col we need
    data_i_df = data_i[col_names_3]

    # 现在数据是无重复版本的，用'cnt_n_2'记录了这个record重复出现的次数
    # New added: Rearrange the data according to their length
    data_lenth = data_i_df.shape[0]         # Data length
    dele_idx = []                           # Idx to be deleted

    if(data_lenth >= 0):
        j = 0
        while (j < data_lenth):
            # N值-1恰好是4倍
            if((data_i_df.iloc[j,0]-1)%5 == 0):
                k = j+1    # 用k记录位置
                # k不是data最后一个值 and [k]在+4的范围内
                while((k < data_lenth) and (data_i_df.iloc[j,0]+5 > data_i_df.iloc[k,0])):
                    # 更新[j]的prob值和cnt值：叠加
                    data_i_df.iloc[j,1] += data_i_df.iloc[k,1]
                    data_i_df.iloc[j,2] += data_i_df.iloc[k,2]
                    # 删除[k]
                    dele_idx.append(k)
                    k += 1
                # j 从 k（其实是k+1）的位置继续
                j = k
            # N值不是4倍且未被并入任何一个已存在的开头
            else:
                # 原地修改（减小）成一个新的区间的开头,
                while((data_i_df.iloc[j,0]-1) %5 != 0):
                    data_i_df.iloc[j,0] -= 1
                # 注意不需要j+1，下一次循环从当前开始，check后面的n需不需要并进来
                # j += 1

    ######### 如果只想保存改动的data，把下面都拿到上面的if里面来
    # Save i_th training data file
    # save_i_trainfile(i)
    # Dele
    data_i_df = data_i_df.drop(dele_idx,axis = 0).copy()

    # Reconstruct and repeat data according to 'cnt_n_2'
    data_i_df = data_i_df.loc[data_i_df.index.repeat(data_i_df['cnt_n_2'])]
    data_i_df = data_i_df[col_names_2]

    # Get the output path
    target_output_path = target_output_head+str(i).zfill(4)+target_output_tail
    data_i_df.to_csv(target_output_path,header=True,index=False,encoding="utf-8")

# clear_output()
print("Done")

100%|██████████| 1196/1196 [00:17<00:00, 68.23it/s] 

Done


- 方法二：read in DA后的data （with 'cnt'）

In [18]:
# Read in
target_path_DA = r"../data/targets_DA"
target_all_path = os.listdir(target_path_DA)

def get_data_i_path(index):
    return os.path.join(target_path_DA,target_all_path[index])

In [19]:
# 粒度为5
for i in tqdm(range(len(target_all_path))):
# for i in tqdm(range(0,10)):

    # Get i_th data path
    path_i = get_data_i_path(i)
    # Get the data
    data_i = pd.read_csv(path_i,encoding="utf-8")
    # Sort data according to 'N'
    data_i.sort_values(by='N',ignore_index=True,inplace=True)
    # Drop duplicates
    data_i_df = data_i.drop_duplicates(inplace = False,ignore_index=True)

    # 现在数据是无重复版本的，'cnt'记录了这个record重复出现的次数
    # New added: Rearrange the data according to their length
    data_lenth = data_i_df.shape[0]         # Data length
    dele_idx = []                           # Idx to be deleted

    if(data_lenth >= 0):
        j = 0
        while (j < data_lenth):
            # N值-1恰好是5倍
            if((data_i_df.iloc[j,0]-1)%5 == 0):
                k = j+1    # 用k记录位置
                # k不是data最后一个值 and [k]在+5的范围内
                while((k < data_lenth) and (data_i_df.iloc[j,0]+5 > data_i_df.iloc[k,0])):
                    # 更新[j]的prob值和cnt值：叠加
                    data_i_df.iloc[j,1] += data_i_df.iloc[k,1]
                    data_i_df.iloc[j,2] += data_i_df.iloc[k,2]
                    # 删除[k]
                    dele_idx.append(k)
                    k += 1
                # j 从 k（其实是k+1）的位置继续
                j = k
            # N值不是5倍且未被并入任何一个已存在的开头
            else:
                # 原地修改（减小）成一个新的区间的开头,
                while((data_i_df.iloc[j,0]-1) %5 != 0):
                    data_i_df.iloc[j,0] -= 1
                # 注意不需要j+1，下一次循环从当前开始，check后面的n需不需要并进来
                # j += 1

    ######### 如果只想保存改动的data，把下面都拿到上面的if里面来
    # Save i_th training data file
    # save_i_trainfile(i)
    # Dele
    data_i_df = data_i_df.drop(dele_idx,axis = 0).copy()

    # Reconstruct and repeat data according to 'cnt_n_2'
    data_i_df = data_i_df.loc[data_i_df.index.repeat(data_i_df['cnt'])]
    data_i_df = data_i_df[col_names_2]

    # Get the output path
    target_output_path = target_output_head+str(i).zfill(4)+target_output_tail
    data_i_df.to_csv(target_output_path,header=True,index=False,encoding="utf-8")

# clear_output()
print("Done")

100%|██████████| 1196/1196 [00:09<00:00, 121.48it/s]

Done


## 2.2 写成cdf
1. 先读入某个targets data，然后写成cdf

In [8]:
# Read in
target_path = r"../data/targets_5"
target_all_path = os.listdir(target_path)
len_target = len(target_all_path)
# Output path
target_output_head= "../data/targets_5_cdf/target_data_NP_"

for i in tqdm(range(0,len_target)):
        target_path_i_path = os.path.join(target_path,target_all_path[i])
        target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

        # Drop the duplicate
        target_df.drop_duplicates(inplace=True)
        # Add and transform P into cdf value
        for j in range(1,target_df.shape[0]):
            target_df.iloc[j,1] += target_df.iloc[j-1,1]

        # Get the output path
        target_output_path = target_output_head+str(i).zfill(4)+target_output_tail
        target_df.to_csv(target_output_path,header=True,index=False,encoding="utf-8")
print("Done")

100%|██████████| 1196/1196 [00:06<00:00, 197.93it/s]

Done



# 3 save
1. 保存结果以及viz环境

In [22]:
list(range(0,10,2))

[0, 2, 4, 6, 8]

In [ ]:
# 保存整个环境
if(plot_flag):
    viz.save(envs=[env_str])